In [2]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

In [3]:
## OBTENEMOS TODA LA INFORMACIÓN DEL SET DE ENTRENAMIENTO.
training = pd.read_csv('labels_training_set.csv', encoding = 'utf-8')
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES EVENTOS.
eventos = pd.read_csv('events_up_to_01062018.csv', encoding = 'utf-8')
## OBTENEMOS TODA LA INFORMACIÓN A TESTEAR.
test = pd.read_csv('trocafone_kaggle_test.csv', encoding = 'utf-8')

In [4]:
## PASAMOS LAS COLUMNAS QUE TIENEN UNA CANTIDAD DE VALORES LIMITADA A UN TIPO CATEGORY
eventos['event'] = eventos['event'].astype('category')
eventos['condition'] = eventos['condition'].astype('category')
eventos['storage'] = eventos['storage'].astype('category')
eventos['search_engine'] = eventos['search_engine'].astype('category')
eventos['channel'] = eventos['channel'].astype('category')
eventos['new_vs_returning'] = eventos['new_vs_returning'].astype('category')
eventos['device_type'] = eventos['device_type'].astype('category')
eventos['color'] = eventos['color'].astype('category')
eventos['region'] = eventos['region'].astype('category')
eventos['country'] = eventos['country'].astype('category')
eventos['operating_system_version'] = eventos['operating_system_version'].astype('category')
eventos['city'] = eventos['city'].astype('category')
eventos['browser_version'] = eventos['browser_version'].astype('category')
eventos['screen_resolution'] = eventos['screen_resolution'].astype('category')
eventos['timestamp'] = pd.to_datetime(eventos['timestamp'], infer_datetime_format=True)

In [5]:
## ORDENAMOS LOS DATOS ṔOR PERSONAS EN PRIMER LUGAR Y TIEMPO EN SEGUNDO.
eventos.sort_values(['person', 'timestamp'], ascending=[True, True], inplace=True)
## DIFERENCIAMOS EN TRES COLUMNAS DIFERENTES EL DIA, MES Y AÑO.
eventos['mes'] = eventos['timestamp'].dt.month
eventos['dia'] = eventos['timestamp'].dt.day
eventos['hora'] = eventos['timestamp'].dt.hour
## ARMAMOS UNA COLUMNA PARA EL DÍA DE LA SEMANA COMO NOMBRE.
eventos['diasemana'] = eventos['timestamp'].dt.weekday_name
## PONEMOS LOS NOMBRES DE MANERA MÁS PROLIJA PARA LOS GRÁFICOS.
eventos.loc[eventos.diasemana.str.contains('Monday', na=False), 'diasemana'] = 'lunes'
eventos.loc[eventos.diasemana.str.contains('Tuesday', na=False), 'diasemana'] = 'martes'
eventos.loc[eventos.diasemana.str.contains('Wednesday', na=False), 'diasemana'] = 'miercoles'
eventos.loc[eventos.diasemana.str.contains('Thursday', na=False), 'diasemana'] = 'jueves'
eventos.loc[eventos.diasemana.str.contains('Friday', na=False), 'diasemana'] = 'viernes'
eventos.loc[eventos.diasemana.str.contains('Saturday', na=False), 'diasemana'] = 'sabado'
eventos.loc[eventos.diasemana.str.contains('Sunday', na=False), 'diasemana'] = 'domingo'
# DEFINIMOS UNA LÓGICA PARA INDICAR SI EL DÍA EN QUE SE EJECUTA EL EVENTO ES FIN DE SEMANA.
eventos['finde'] = 0
eventos.loc[(eventos.diasemana.str.contains('DOM', na=False) | eventos.diasemana.str.contains('SAB', na=False)), 'finde'] = 1
# DEFINIMOS EL MES COMO NOMBRE PARA FACILITAR LAS COLUMNAS
eventos['mesMayus'] = ''
eventos.loc[eventos.mes == 1, 'mesMayus'] = 'enero'
eventos.loc[eventos.mes == 2, 'mesMayus'] = 'febrero'
eventos.loc[eventos.mes == 3, 'mesMayus'] = 'marzo'
eventos.loc[eventos.mes == 4, 'mesMayus'] = 'abril'
eventos.loc[eventos.mes == 5, 'mesMayus'] = 'mayo'
eventos.loc[eventos.mes == 6, 'mesMayus'] = 'junio'
eventos.loc[eventos.mes == 7, 'mesMayus'] = 'julio'
eventos.loc[eventos.mes == 8, 'mesMayus'] = 'agosto'
eventos.loc[eventos.mes == 9, 'mesMayus'] = 'septiembre'
eventos.loc[eventos.mes == 10, 'mesMayus'] = 'octubre'
eventos.loc[eventos.mes == 11, 'mesMayus'] = 'noviembre'
eventos.loc[eventos.mes == 12, 'mesMayus'] = 'diciembre'
# ARMAMOS UNA LÓGICA PARA SEGMENTAR LAS FRANJAS HORARIAS.
# MADRUGADA de 00 a 06
eventos['hora_madrugada'] = 0
eventos.loc[((eventos.hora > -1) & (eventos.hora < 7)), 'hora_madrugada'] = 1
# MAÑANA de 07 a 11
eventos['hora_mañana'] = 0
eventos.loc[((eventos.hora > 6) & (eventos.hora < 12)), 'hora_mañana'] = 1
# ALMUERZO de 12 a 13
eventos['hora_almuerzo'] = 0
eventos.loc[((eventos.hora > 11) & (eventos.hora < 14)), 'hora_almuerzo'] = 1
# TARDE de 14 a 18
eventos['hora_tarde'] = 0
eventos.loc[((eventos.hora > 13) & (eventos.hora < 19)), 'hora_tarde'] = 1
# NOCHE de 19 a 23
eventos['hora_noche'] = 0
eventos.loc[((eventos.hora > 18) & (eventos.hora < 24)), 'hora_noche'] = 1
# TRANSFORMAMOS EN CATEGÓRICAS EL DÍA DE LA SEMANA Y EL MES.
eventos['diasemana'] = eventos['diasemana'].astype('category')
eventos['mesMayus'] = eventos['mesMayus'].astype('category')

In [7]:
dummies_diasemana = pd.get_dummies(eventos['diasemana'], drop_first=False)

eventos = pd.concat([eventos, dummies_diasemana], axis=1)

dummies_mesMayus = pd.get_dummies(eventos['mesMayus'], drop_first=False)

eventos = pd.concat([eventos, dummies_mesMayus], axis=1)

In [8]:
train_completo = pd.merge(training, eventos, on='person', how='left')
train_completo.dropna(subset=['person']) 
train_completo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1171886 entries, 0 to 1171885
Data columns (total 47 columns):
person                      1171886 non-null object
label                       1171886 non-null int64
timestamp                   1171886 non-null datetime64[ns]
event                       1171886 non-null category
url                         94875 non-null object
sku                         665336 non-null float64
model                       665767 non-null object
condition                   665336 non-null category
storage                     665336 non-null category
color                       665336 non-null category
skus                        249587 non-null object
search_term                 55774 non-null object
staticpage                  5660 non-null object
campaign_source             94940 non-null object
search_engine               52829 non-null category
channel                     102299 non-null category
new_vs_returning            102299 non-null category


In [9]:
y = pd.factorize(train_completo['label'])[0]
train_completo = train_completo[['mes','dia','hora','finde','hora_madrugada','hora_mañana','hora_almuerzo','hora_tarde','hora_noche','domingo','jueves','lunes','martes','miercoles','sabado','viernes','abril','enero','febrero','marzo','mayo']]
# VER COMO OBTENER LAS COLUMNAS 
# ESPECÍFICAS QUE QUEREMOS TRABAJAR.
features = train_completo.columns

In [10]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [11]:
# Creamos un clasificador con Random Forest..
clf = RandomForestClassifier(n_jobs=2, random_state=0)
# Entrenamos.
clf.fit(train_completo[features], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [12]:
# Predecimos.
eventos['label'] = 0
clf.predict(eventos[features])

array([0, 0, 0, ..., 0, 0, 0])

In [13]:
# REVISAR. todos dan iguales.
prueba = clf.predict_proba(eventos[features])[0:1000]

In [14]:
y = np.unique(prueba)

In [15]:
y

array([0.01235825, 0.01812917, 0.01849175, 0.01875001, 0.01965383,
       0.02193093, 0.02428836, 0.02641002, 0.03184322, 0.03529104,
       0.03785505, 0.03791787, 0.03998886, 0.04099342, 0.04166749,
       0.04365776, 0.04381605, 0.04452895, 0.04941189, 0.05207888,
       0.05265529, 0.05560323, 0.05745559, 0.05766315, 0.05879958,
       0.05919772, 0.06007577, 0.06078276, 0.06098996, 0.0643176 ,
       0.06491026, 0.06661598, 0.06685586, 0.06872183, 0.06980955,
       0.07035905, 0.07192336, 0.07444623, 0.07797967, 0.08167662,
       0.08196785, 0.08614193, 0.08759783, 0.08912079, 0.09256653,
       0.094044  , 0.09643218, 0.10937078, 0.11211194, 0.1236754 ,
       0.13955269, 0.14376049, 0.15597018, 0.15709843, 0.16441547,
       0.17199945, 0.22962318, 0.33656206, 0.66343794, 0.77037682,
       0.82800055, 0.83558453, 0.84290157, 0.84402982, 0.85623951,
       0.86044731, 0.8763246 , 0.88788806, 0.89062922, 0.90356782,
       0.905956  , 0.90743347, 0.91087921, 0.91240217, 0.91385